# Create trip statistics

# Purpose
Before looking at the dynamics of the ferries from the time series it is a good idea to first look at some longer term trends. Statistics for each trip will be generated and saved as a first data reduction, to spot trends over the day/week/month and year.

# Methodology
* Trip statistics will be generated for each trip containing:
   * for all columns: min/mean/max/median/std
   * energy consumption for all thrusters
* The statistics will be stored into a [xarray](http://xarray.pydata.org/en/stable/)

# Setup

In [ ]:
# %load imports.py
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize
from src.data import get_dataset
from src.data import trips
import scipy.integrate
import seaborn as sns
import xarray as xr


In [ ]:
%%time
df = get_dataset.get(n_rows=None)
deltas = ['delta_%i' % i for i in range(1,5)]
df.drop(columns=deltas, inplace=True)
df['trip_time'] = pd.TimedeltaIndex(df['trip_time']).total_seconds()

mask = df['reversing'].copy()
df['reversing'].loc[mask] = 1
df['reversing'].loc[~mask] = 0


In [ ]:
df.head()

In [ ]:
groups = df.groupby(by='trip_no')
assert (groups.last()['sog'] < 0.3).all()

In [ ]:
trip = df.groupby(by='trip_no').get_group(11)

In [ ]:
visualize.plot_map(trip)

In [ ]:
def integrate_time(trip):
    trip_ = trip.copy()
    t = pd.TimedeltaIndex(trip_['trip_time'],unit='s').total_seconds()
    trip_.drop(columns=['trip_time'], inplace=True)
    integral_trip = scipy.integrate.simps(y=trip_.T,x=t)
    s = pd.Series(data=integral_trip, name='integral', index=trip_.columns)
    
    return s

In [ ]:
integrate_time(trip)

In [ ]:
t = pd.TimedeltaIndex(trip['trip_time'], unit='s').total_seconds()
scipy.integrate.simps(y=trip['power_em_thruster_total'],x=t)

In [ ]:
def trip_statistic(trip):
    
    stats = trip.describe()  # General statistics
    integral_trip = integrate_time(trip)
    stats = stats.append(integral_trip)
    return stats

In [ ]:

ds_stats = None
for trip_no, trip in df.groupby(by='trip_no'):
    

    trip_ = trip.copy()
    trip_direction = trip_.iloc[0]['trip_direction']
    #trip_.drop(columns=['trip_no','trip_direction'], inplace=True)
    trip_.drop(columns=['trip_no'], inplace=True)
    
    
    stats = trip_statistic(trip_)
    stats.index.name = 'statistic'
    
    ds = xr.Dataset.from_dataframe(stats)
    ds = ds.expand_dims('trip_no')
    ds = ds.assign_coords(trip_no=np.array([trip_no],dtype=np.int64))
    #ds.attrs['trip_direction'] = trip_direction
    
    if ds_stats is None:
        ds_stats = ds
    else:
        ds_stats = xr.concat([ds_stats,ds], dim="trip_no")


In [ ]:
ds_stats

In [ ]:
ds_stats.coords['statistic']

In [ ]:
ds

In [ ]:
ds_stats.sel(trip_no=2, statistic='mean')

In [ ]:
ds_stats.sel(statistic='mean').plot.scatter(x="sog",y="trip_time")

In [ ]:
ds_stats.sel(statistic='max').plot.scatter(x="sog",y="power_em_thruster_total")

In [ ]:
ds_stats.sel(statistic=['min','mean','max']).plot.scatter(x="sog",y="power_em_thruster_total", hue='statistic');


In [ ]:
xr.plot.hist(ds_stats.sel(statistic='mean')['sog'], bins=20);

In [ ]:
xr.plot.hist(ds_stats.sel(statistic='integral')["power_em_thruster_total"], bins=20);

In [ ]:
ds_stats.sel(statistic='integral').plot.scatter(x="sog",y="power_em_thruster_total")

In [ ]:
df_mean = ds_stats.sel(statistic='mean').to_dataframe()

In [ ]:
df_means = df_mean.groupby(by='trip_direction').mean()
df_stds =  df_mean.groupby(by='trip_direction').std()

In [ ]:

directions = pd.Series({
    0 : 'Helsingör-Helsinborg',
    1 : 'Helsinborg-Helsingör',
})


x = directions[df_means.index]

fig,ax=plt.subplots()
ax.bar(x=x, height=df_means['power_em_thruster_total'], yerr=df_stds['power_em_thruster_total'])

fig,ax=plt.subplots()
ax.bar(x=x, height=df_means['trip_time'], yerr=df_stds['trip_time'])

fig,ax=plt.subplots()
ax.bar(x=x, height=df_means['sog'], yerr=df_stds['sog'])

## Save statistics

In [ ]:
df_mean = ds_stats.sel(statistic='mean').to_dataframe()
df_integral = ds_stats.sel(statistic='integral').to_dataframe()
df_std = ds_stats.sel(statistic='std').to_dataframe()

df_max = ds_stats.sel(statistic='max').to_dataframe()
df_stats = df_mean.drop(columns=['statistic'])

In [ ]:
df_ = df.reset_index()
start_times = df_.groupby('trip_no').first()['time']
end_time = df_.groupby('trip_no').last()['time']


In [ ]:
integral_columns = ['power_em_thruster_%i' %i for i in range(1,5)]
integral_columns+=['power_em_thruster_total','power_heeling']
df_stats[integral_columns] = df_integral[integral_columns]

max_columns = ['trip_time']
df_stats[max_columns] = df_max[max_columns]
df_stats['start_time'] = start_times
df_stats['end_time'] = end_time

In [ ]:
df_stats.head()

In [ ]:
from azureml.core import Workspace, Dataset

subscription_id = '3e9a363e-f191-4398-bd11-d32ccef9529c'
resource_group = 'demops'
workspace_name = 'D2E2F'

workspace = Workspace(subscription_id, resource_group, workspace_name)

def save():
    
    df_stats_save = df_stats.copy()
    df_stats_save.reset_index(inplace=True)
    
    datastore = workspace.get_default_datastore()
    dataset_2 = Dataset.Tabular.register_pandas_dataframe(dataframe=df_stats_save, target=datastore, name=new_name)


In [ ]:
new_name = 'tycho_short_statistics'
if not new_name in workspace.datasets:
    save()
    

In [ ]:
columns = ['cos_pm%i' % i for i in range(1,5)]
df_std.mean()[columns]

In [ ]:
columns = ['sin_pm%i' % i for i in range(1,5)]
df_std.mean()[columns]